# Keras Tuner

In [20]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

In [2]:
(img_train, label_train), (img_test, lable_test)= keras.datasets.fashion_mnist.load_data()

In [3]:
img_train.shape

(60000, 28, 28)

In [4]:
img_test.shape

(10000, 28, 28)

In [5]:
import numpy as np

In [6]:
np.min(img_train)

0

In [7]:
np.max(img_train)

255

In [8]:
np.min(img_test)

0

In [9]:
np.max(img_test)

255

In [10]:
img_train = img_train.astype('float32')/255.0

In [11]:
img_test = img_test.astype('float32')/255.0

In [12]:
np.min(img_train)

0.0

In [13]:
np.max(img_train)

1.0

In [17]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    hp_units = hp.Int('units', min_value=32,
                     max_value=512, step=32)
    model.add(keras.layers.Dense(units= hp_units,
                                activation= "relu"))
    model.add(keras.layers.Dense(10))
    hp_learning_rate = hp.Choice('learning_rate',
                                values= [1e-2,1e-3,1e-4])
    
    model.compile(optimizer= keras.optimizers.Adam(
    learning_rate= hp_learning_rate),
                 loss= keras.losses.SparseCategoricalCrossentropy(
                 from_logits=True),
                 metrics = ["accuracy"])
    return model

In [18]:
tuner = kt.Hyperband(model_builder,
                    objective= 'val_accuracy',
                    max_epochs= 10,
                    factor= 3,
                    directory= 'my_dir',
                    project_name= 'my_tuner')

In [21]:
stop_early = tf.keras.callbacks.EarlyStopping(
monitor= 'val_loss', patience=5)

In [24]:
tuner.search(img_train, label_train,
            epochs= 5, validation_split=0.2,
            callbacks = [stop_early])

Trial 32 Complete [00h 00m 19s]
val_accuracy: 0.8649166822433472

Best val_accuracy So Far: 0.890333354473114
Total elapsed time: 00h 06m 36s
INFO:tensorflow:Oracle triggered exit


In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [27]:
print(best_hps.get('units'))

448


In [28]:
print(best_hps.get('learning_rate'))

0.001


In [29]:
model = tuner.hypermodel.build(best_hps)

In [30]:
history = model.fit(img_train, label_train,
                   epochs=5, validation_split= 0.2)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4951 - accuracy: 0.8236 - val_loss: 0.4084 - val_accuracy: 0.8515
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3712 - accuracy: 0.8647 - val_loss: 0.3532 - val_accuracy: 0.8754
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3327 - accuracy: 0.8774 - val_loss: 0.3660 - val_accuracy: 0.8718
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3041 - accuracy: 0.8881 - val_loss: 0.3408 - val_accuracy: 0.8777
Epoch 5/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2878 - accuracy: 0.8934 - val_loss: 0.3288 - val_accuracy: 0.8849


In [31]:
val_acc_per_epoch = history.history['val_accuracy']

In [32]:
print(val_acc_per_epoch)

[0.8514999747276306, 0.8754166960716248, 0.871833324432373, 0.8777499794960022, 0.8849166631698608]
